In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
import tensorflow as tensorflow
device_name = tensorflow.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
from tensorflow.keras.layers import Conv2D, Dense, BatchNormalization, Activation
from tensorflow.keras.layers import add, Add

# identity block은 shortcut 단에 conv layer가 없는 block 영역
def identity_block(input_tensor, middle_kernel_size, filters, stage, block):
    '''
    함수 입력 인자 설명
    input_tensor : 입력 tensor

    middle_kernel_size : 중간에 위치하는 kernel 크기. identity block내에 있는 두개의 conv layer중 1x1 kernel이 아니고, 3x3 kernel임. 
    (3x3 커널이 이외에도 5x5 kernel도 지정할 수 있게 구성)

    filters: 3개 conv layer들의 filter개수를 list 형태로 입력 받음. 첫번째 원소는 첫번째 1x1 filter 개수, 두번째는 3x3 filter 개수, 세번째는 마지막 1x1 filter 개수

    stage: identity block들이 여러개가 결합되므로 이를 구분하기 위해서 설정. 동일한 filter수를 가지는 identity block들을  동일한 stage로 설정.  

    block: 동일 stage내에서 identity block을 구별하기 위한 구분자
    ''' 
    
    # filters : filter 개수를 각각 filter1, filter2, filter3 list 형태로 할당. 
    # filter은 첫번째 1x1 filter 개수
    # filter2는 3x3 filter 개수
    # filter3는 마지막 1x1 filter 개수
    filter1, filter2, filter3 = filters
    
    # conv layer와 Batch normalization layer각각에 고유한 이름을 부여하기 위해 설정
    # 입력받은 stage와 block에 기반하여 이름 부여
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # 이전 layer에 입력 받은 input_tensor를 기반으로 첫번째 1x1 Conv->Batch Norm->Relu 수행. 
    # 첫번째 1x1 Conv에서 Channel Dimension Reduction(1/4) 수행
    x = Conv2D(filters=filter1, kernel_size=(1, 1), kernel_initializer='he_normal', name=conv_name_base+'2a')(input_tensor)
    x = BatchNormalization(axis=3, name=bn_name_base+'2a')(x)
    x = Activation('relu')(x)
    
    # 두번째 3x3 Conv->Batch Norm->ReLU 수행
    # 3x3이 아닌 다른 kernel size도 구성 가능할 수 있도록 identity_block() 인자로 입력받은 middle_kernel_size를 이용
    # Conv 수행 출력 사이즈가 변하지 않도록 padding='same'으로 설정
    # filter 개수는 이전의 1x1 filter개수와 동일
    x = Conv2D(filters=filter2, kernel_size=middle_kernel_size, padding='same', kernel_initializer='he_normal', name=conv_name_base+'2b')(x)
    x = BatchNormalization(axis=3, name=bn_name_base+'2b')(x)
    x = Activation('relu')(x)
    
    # 마지막 1x1 Conv->Batch Norm 수행
    # ReLU를 수행 X (input tensor 더한 이후에 ReLU 적용)
    # filter 크기는 input_tensor channel 차원 개수로 복구
    x = Conv2D(filters=filter3, kernel_size=(1, 1), kernel_initializer='he_normal', name=conv_name_base+'2c')(x)
    x = BatchNormalization(axis=3, name=bn_name_base+'2c')(x)

    # Residual Block 수행 결과 & input_tensor를 합 (Skip Connection)
    x = Add()([input_tensor, x])

    # 최종 ReLU 적용
    x = Activation('relu')(x)
    
    return x